In [26]:
import os
import sys
import numpy as np
import pandas as pd
from tqdm import tqdm
import joblib

## Preprocessing
* Change input data (ex. train.txt) into CRF model input format (ex. train.data)
    * CRF model input format (ex. train.data):
        ```
        肝 O
        功 O
        能 O
        6 B-med_exam
        8 I-med_exam
        ```

In [2]:
file_path='./data/bind.txt'

In [3]:
def loadInputFile(path):
    trainingset = list()  # store trainingset [content,content,...]
    position = list()  # store position [article_id, start_pos, end_pos, entity_text, entity_type, ...]
    mentions = dict()  # store mentions[mention] = Type
    with open(file_path, 'r', encoding='utf8') as f:
        file_text=f.read().encode('utf-8').decode('utf-8-sig')
    datas=file_text.split('\n\n--------------------\n\n')[:-1]
    for data in datas:
        data=data.split('\n')
        content=data[0]
        trainingset.append(content)
        annotations=data[1:]
        for annot in annotations[1:]:
            annot=annot.split('\t') #annot= article_id, start_pos, end_pos, entity_text, entity_type
            position.extend(annot)
            mentions[annot[3]]=annot[4]
    
    return trainingset, position, mentions

In [4]:
def CRFFormatData(trainingset, position, path):
    if (os.path.isfile(path)):
        os.remove(path)
    outputfile = open(path, 'a', encoding= 'utf-8')

    # output file lines
    count = 0 # annotation counts in each content
    tagged = list()
    for article_id in range(len(trainingset)):
        trainingset_split = list(trainingset[article_id])
        while '' or ' ' in trainingset_split:
            if '' in trainingset_split:
                trainingset_split.remove('')
            else:
                trainingset_split.remove(' ')
        start_tmp = 0
        for position_idx in range(0,len(position),5):
            if int(position[position_idx]) == article_id:
                count += 1
                if count == 1:
                    start_pos = int(position[position_idx+1])
                    end_pos = int(position[position_idx+2])
                    entity_type=position[position_idx+4]
                    if start_pos == 0:
                        token = list(trainingset[article_id][start_pos:end_pos])
                        whole_token = trainingset[article_id][start_pos:end_pos]
                        for token_idx in range(len(token)):
                            if len(token[token_idx].replace(' ','')) == 0:
                                continue
                            # BIO states
                            if token_idx == 0:
                                label = 'B-'+entity_type
                            else:
                                label = 'I-'+entity_type
                            
                            output_str = token[token_idx] + ' ' + label + '\n'
                            outputfile.write(output_str)

                    else:
                        token = list(trainingset[article_id][0:start_pos])
                        whole_token = trainingset[article_id][0:start_pos]
                        for token_idx in range(len(token)):
                            if len(token[token_idx].replace(' ','')) == 0:
                                continue
                            
                            output_str = token[token_idx] + ' ' + 'O' + '\n'
                            outputfile.write(output_str)

                        token = list(trainingset[article_id][start_pos:end_pos])
                        whole_token = trainingset[article_id][start_pos:end_pos]
                        for token_idx in range(len(token)):
                            if len(token[token_idx].replace(' ','')) == 0:
                                continue
                            # BIO states
                            if token[0] == '':
                                if token_idx == 1:
                                    label = 'B-'+entity_type
                                else:
                                    label = 'I-'+entity_type
                            else:
                                if token_idx == 0:
                                    label = 'B-'+entity_type
                                else:
                                    label = 'I-'+entity_type

                            output_str = token[token_idx] + ' ' + label + '\n'
                            outputfile.write(output_str)

                    start_tmp = end_pos
                else:
                    start_pos = int(position[position_idx+1])
                    end_pos = int(position[position_idx+2])
                    entity_type=position[position_idx+4]
                    if start_pos<start_tmp:
                        continue
                    else:
                        token = list(trainingset[article_id][start_tmp:start_pos])
                        whole_token = trainingset[article_id][start_tmp:start_pos]
                        for token_idx in range(len(token)):
                            if len(token[token_idx].replace(' ','')) == 0:
                                continue
                            output_str = token[token_idx] + ' ' + 'O' + '\n'
                            outputfile.write(output_str)

                    token = list(trainingset[article_id][start_pos:end_pos])
                    whole_token = trainingset[article_id][start_pos:end_pos]
                    for token_idx in range(len(token)):
                        if len(token[token_idx].replace(' ','')) == 0:
                            continue
                        # BIO states
                        if token[0] == '':
                            if token_idx == 1:
                                label = 'B-'+entity_type
                            else:
                                label = 'I-'+entity_type
                        else:
                            if token_idx == 0:
                                label = 'B-'+entity_type
                            else:
                                label = 'I-'+entity_type
                        
                        output_str = token[token_idx] + ' ' + label + '\n'
                        outputfile.write(output_str)
                    start_tmp = end_pos

        token = list(trainingset[article_id][start_tmp:])
        whole_token = trainingset[article_id][start_tmp:]
        for token_idx in range(len(token)):
            if len(token[token_idx].replace(' ','')) == 0:
                continue

            
            output_str = token[token_idx] + ' ' + 'O' + '\n'
            outputfile.write(output_str)

        count = 0
    
        output_str = '\n'
        outputfile.write(output_str)
        ID = trainingset[article_id]

        if article_id%10 == 0:
            print('Total complete articles:', article_id)

    # close output file
    outputfile.close()

In [5]:
trainingset, position, mentions=loadInputFile(file_path)

In [6]:
data_path='./data/bind_mark.data'
CRFFormatData(trainingset, position, data_path)

Total complete articles: 0
Total complete articles: 10
Total complete articles: 20
Total complete articles: 30
Total complete articles: 40
Total complete articles: 50
Total complete articles: 60
Total complete articles: 70
Total complete articles: 80
Total complete articles: 90
Total complete articles: 100
Total complete articles: 110
Total complete articles: 120
Total complete articles: 130
Total complete articles: 140


## NER model
### CRF (Conditional Random Field model)
* Using `sklearn-crfsuite` API

    (you may try `CRF++`, `python-crfsuite`, `pytorch-crfsuite`(neural network version))

In [7]:
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics
from sklearn_crfsuite.metrics import flat_classification_report

In [8]:
def CRF(x_train, y_train, x_test, y_test):
    crf = sklearn_crfsuite.CRF(
        algorithm='lbfgs',
        c1=0.1,
        c2=0.1,
        max_iterations=100,
        all_possible_transitions=True
    )
    crf.fit(x_train, y_train)
    # print(crf)
    y_pred = crf.predict(x_test)
    y_pred_mar = crf.predict_marginals(x_test)

    # print(y_pred_mar)

    labels = list(crf.classes_)
    labels.remove('O')
    f1score = metrics.flat_f1_score(y_test, y_pred, average='weighted', labels=labels)
    sorted_labels = sorted(labels,key=lambda name: (name[1:], name[0])) # group B and I results
    print(flat_classification_report(y_test, y_pred, labels=sorted_labels, digits=3))
    return y_pred, y_pred_mar, f1score

## Model Input: 
* input features:
    * word vector: pretrained traditional chinese word embedding by Word2Vec-CBOW
    
    (you may try add some other features, ex. pos-tag, word_length, word_position, ...) 

In [9]:
import numpy as np

In [10]:
# load pretrained word vectors
# get a dict of tokens (key) and their pretrained word vectors (value)
# pretrained word2vec CBOW word vector: https://fgc.stpi.narl.org.tw/activity/videoDetail/4b1141305ddf5522015de5479f4701b1
dim = 0
word_vecs= {}
# open pretrained word vector file
with open('cna.cbow.cwe_p.tar_g.512d.0.txt', encoding='utf-8') as f:
    for line in f:
        tokens = line.strip().split()

        # there 2 integers in the first line: vocabulary_size, word_vector_dim
        if len(tokens) == 2:
            dim = int(tokens[1])
            continue
    
        word = tokens[0] 
        vec = np.array([ float(t) for t in tokens[1:] ])
        word_vecs[word] = vec

In [11]:
print('vocabulary_size: ',len(word_vecs),' word_vector_dim: ',vec.shape)

vocabulary_size:  158566  word_vector_dim:  (512,)


Here we split data into training dataset and testing dataset,
however, we'll provide `development data` and `test data` which is real testing dataset.

You should upload prediction on `development data` and `test data` to system, not this splitted testing dataset.

In [12]:
# load `train.data` and separate into a list of labeled data of each text
# return:
#   data_list: a list of lists of tuples, storing tokens and labels (wrapped in tuple) of each text in `train.data`
#   traindata_list: a list of lists, storing training data_list splitted from data_list
#   testdata_list: a list of lists, storing testing data_list splitted from data_list
from sklearn.model_selection import train_test_split
def Dataset(data_path):
    with open(data_path, 'r', encoding='utf-8') as f:
        data=f.readlines()#.encode('utf-8').decode('utf-8-sig')
    data_list, data_list_tmp = list(), list()
    article_id_list=list()
    idx=0
    for row in data:
        data_tuple = tuple()
        if row == '\n':
            article_id_list.append(idx)
            idx+=1
            data_list.append(data_list_tmp)
            data_list_tmp = []
        else:
            row = row.strip('\n').split(' ')
            data_tuple = (row[0], row[1])
            data_list_tmp.append(data_tuple)
    if len(data_list_tmp) != 0:
        data_list.append(data_list_tmp)
    
    # here we random split data into training dataset and testing dataset
    # but you should take `development data` or `test data` as testing data
    # At that time, you could just delete this line, 
    # and generate data_list of `train data` and data_list of `development/test data` by this function
#     traindata_list, testdata_list, traindata_article_id_list, testdata_article_id_list=train_test_split(data_list,
#                                                                                                     article_id_list,
#                                                                                                     test_size=0.33,
#                                                                                                     random_state=42)
    
    return data_list, article_id_list

In [13]:
# look up word vectors
# turn each word into its pretrained word vector
# return a list of word vectors corresponding to each token in train.data
def Word2Vector(data_list, embedding_dict):
    embedding_list = list()

    # No Match Word (unknown word) Vector in Embedding
    unk_vector=np.random.rand(*(list(embedding_dict.values())[0].shape))

    for idx_list in range(len(data_list)):
        embedding_list_tmp = list()
        for idx_tuple in range(len(data_list[idx_list])):
            key = data_list[idx_list][idx_tuple][0] # token

            if key in embedding_dict:
                value = embedding_dict[key]
            else:
                value = unk_vector
            embedding_list_tmp.append(value)
        embedding_list.append(embedding_list_tmp)
    return embedding_list

In [14]:
# input features: pretrained word vectors of each token
# return a list of feature dicts, each feature dict corresponding to each token
def Feature(embed_list):
    feature_list = list()
    for idx_list in range(len(embed_list)):
        feature_list_tmp = list()
        for idx_tuple in range(len(embed_list[idx_list])):
            feature_dict = dict()
            for idx_vec in range(len(embed_list[idx_list][idx_tuple])):
                feature_dict['dim_' + str(idx_vec+1)] = embed_list[idx_list][idx_tuple][idx_vec]
            feature_list_tmp.append(feature_dict)
        feature_list.append(feature_list_tmp)
    return feature_list

In [15]:
# get the labels of each tokens in train.data
# return a list of lists of labels
def Preprocess(data_list):
    label_list = list()
    for idx_list in range(len(data_list)):
        label_list_tmp = list()
        for idx_tuple in range(len(data_list[idx_list])):
            label_list_tmp.append(data_list[idx_list][idx_tuple][1])
        label_list.append(label_list_tmp)
    return label_list

## Training

In [16]:
data_path='./data/bind_mark.txt'

In [17]:
data_list, article_id_list = Dataset(data_path)

In [18]:
# Load Word Embedding
trainembed_list = Word2Vector(data_list, word_vecs)
#testembed_list = Word2Vector(testdata_list, word_vecs)

# CRF - Train Data (Augmentation Data)
x_train = Feature(trainembed_list)
y_train = Preprocess(data_list)

# CRF - Test Data (Golden Standard)
# x_test = Feature(testembed_list)
# y_test = Preprocess(testdata_list)

In [50]:
x_train[0][500]

{'dim_1': 0.916807,
 'dim_2': 0.8093,
 'dim_3': -0.999875,
 'dim_4': -0.281618,
 'dim_5': -0.682645,
 'dim_6': 0.292539,
 'dim_7': -0.366005,
 'dim_8': -0.064268,
 'dim_9': -1.005169,
 'dim_10': -0.8995,
 'dim_11': 0.999952,
 'dim_12': -0.001803,
 'dim_13': -0.217045,
 'dim_14': 0.824236,
 'dim_15': -0.992083,
 'dim_16': -0.015651,
 'dim_17': 0.05862,
 'dim_18': 0.255725,
 'dim_19': 0.389857,
 'dim_20': 0.028796,
 'dim_21': 0.303875,
 'dim_22': 0.787222,
 'dim_23': 1.753148,
 'dim_24': -0.815557,
 'dim_25': 0.479956,
 'dim_26': 0.696645,
 'dim_27': -0.981546,
 'dim_28': -0.296531,
 'dim_29': 0.472519,
 'dim_30': -0.151441,
 'dim_31': 0.124761,
 'dim_32': 0.008898,
 'dim_33': 0.924069,
 'dim_34': 0.620232,
 'dim_35': 1.021217,
 'dim_36': 0.178519,
 'dim_37': 0.353136,
 'dim_38': 0.669345,
 'dim_39': 0.828234,
 'dim_40': -0.308989,
 'dim_41': 0.541489,
 'dim_42': -0.157816,
 'dim_43': 0.057364,
 'dim_44': 0.420615,
 'dim_45': -0.008287,
 'dim_46': -1.656629,
 'dim_47': -0.771772,
 'dim_4

In [19]:
# x_test[0][1]

In [20]:
# len(x_test[0])

In [21]:
# y_test[0]

In [38]:
trainembed_list[0][4].shape

(512,)

In [ ]:
len(trainembed_list[0])

In [37]:
x_train[0][0]

{'dim_1': -0.052179,
 'dim_2': -1.332038,
 'dim_3': 0.881332,
 'dim_4': -0.729317,
 'dim_5': -0.9045,
 'dim_6': -2.194207,
 'dim_7': -1.011986,
 'dim_8': 0.823145,
 'dim_9': -0.162411,
 'dim_10': -0.591838,
 'dim_11': -3.565321,
 'dim_12': 2.665407,
 'dim_13': 2.782666,
 'dim_14': -1.015439,
 'dim_15': 1.529348,
 'dim_16': -0.668558,
 'dim_17': 1.61489,
 'dim_18': 3.155598,
 'dim_19': 0.140376,
 'dim_20': -0.673071,
 'dim_21': 3.073506,
 'dim_22': -1.548101,
 'dim_23': -0.436637,
 'dim_24': -1.563263,
 'dim_25': 1.817517,
 'dim_26': -1.525961,
 'dim_27': -2.713351,
 'dim_28': 0.669806,
 'dim_29': -1.245986,
 'dim_30': 0.337858,
 'dim_31': -1.783374,
 'dim_32': 1.091606,
 'dim_33': 0.983019,
 'dim_34': 2.41405,
 'dim_35': 1.955675,
 'dim_36': -3.083815,
 'dim_37': -2.323624,
 'dim_38': 1.263534,
 'dim_39': 1.226918,
 'dim_40': -1.817853,
 'dim_41': 0.901008,
 'dim_42': -0.681898,
 'dim_43': 2.544874,
 'dim_44': 1.041264,
 'dim_45': 1.893114,
 'dim_46': 1.043241,
 'dim_47': 0.368856,
 'd

In [30]:
type(x_train[0])

list

In [22]:
def CRF2(X_train, y_train):
    crf = sklearn_crfsuite.CRF(
        algorithm='lbfgs',
        c1=0.1,
        c2=0.1,
        max_iterations=100,
        all_possible_transitions=True
    )
    crf.fit(X_train, y_train)
    
    return crf

In [23]:
crf = CRF2(x_train, y_train)

In [27]:
joblib.dump(crf, './data/model_crf.joblib')

['./data/model_crf.joblib']

In [56]:
###################################
d_path = './data/development_1.txt'
def dataprocess_no_matrix2(path):
    datalist=[]
    with open(path, 'r', encoding="utf-8") as f:
        x = ''
        for line in f:
            if line != '--------------------\n':
                x = x + line + '\n'
            else:
                datalist.append(x)
                x = ''
    
    datadir = {}
    for i, j in enumerate(datalist):
        if i == 0:
            datadir[i] = j.split('\n')[2]
        else:
            datadir[i] = j.split('\n')[4]
    
    return datadir

d_dic = dataprocess_no_matrix2(d_path)

def Word2Vector0(data, word_vecs):
    X_feature = []
    unk_vector=np.random.rand(*(list(word_vecs.values())[0].shape))
    for w in data:
        embedding_dict = {}
        if w in word_vecs:
            value = word_vecs[w]
        else:
            value = unk_vector
        
        for i in range(value.shape[0]):
            embedding_dict['dim_{}'.format(str(i+1))] = value[i]
        
        X_feature.append(embedding_dict)
    return X_feature

X_test = []
for i, j in d_dic.items():
    X_test.append(Word2Vector0(j, word_vecs))
###################################

In [99]:
d_list = []
for i, j in d_dic.items():
    d_list.append(j)

In [100]:
d_id_list = [i for i in range(len(d_list))]

In [57]:
y_pred = crf.predict(X_test)

In [58]:
for i in y_pred:
    for w in i:
        if w != 'O':
            print(w)

B-time
I-time
I-time
B-money
I-money
I-money
I-money


## Output data
* Change model output into `output.tsv` 
* Only accept this output format uploading to competition system

In [105]:
output="article_id\tstart_position\tend_position\tentity_text\tentity_type\n"
for test_id in range(len(y_pred)):
    pos=0
    start_pos=None
    end_pos=None
    entity_text=None
    entity_type=None
    for pred_id in range(len(y_pred[test_id])):
        if pred_id+1 == len(y_pred[test_id]):
            c = 'O'
        else:
            c = y_pred[test_id][pred_id+1][0]
        
        if y_pred[test_id][pred_id][0]=='B':
            start_pos=pos
            entity_type=y_pred[test_id][pred_id][2:]
        elif start_pos is not None and y_pred[test_id][pred_id][0]=='I' and c=='O':
            end_pos=pos
            entity_text=''.join([d_list[test_id][position][0] for position in range(start_pos,end_pos+1)])
            line=str(d_id_list[test_id])+'\t'+str(start_pos)+'\t'+str(end_pos+1)+'\t'+entity_text+'\t'+entity_type
            output+=line+'\n'
        pos+=1     

In [103]:
output_path='output.csv'
with open(output_path,'w',encoding='utf-8') as f:
    f.write(output)

In [106]:
print(output)

article_id	start_position	end_position	entity_text	entity_type
9	757	760	訓了。	time
22	3484	3488	一萬多塊	money



## Note
* You may try `python-crfsuite` to train an neural network for NER tagging optimized by gradient descent back propagation
    * [Documentation](https://github.com/scrapinghub/python-crfsuite)
* You may try `CRF++` tool for NER tagging by CRF model
    * [Documentation](http://taku910.github.io/crfpp/)
    * Need design feature template
    * Can only computed in CPU
* You may try other traditional chinese word embedding (ex. fasttext, bert, ...) for input features
* You may try add other features for NER model, ex. POS-tag, word_length, word_position, ...
* You should upload the prediction output on `development data` or `test data` provided later to the competition system. Note don't upload prediction output on the splitted testing dataset like this baseline example.

-----------------------------------------------------